## Retrieval augmented generation
 
In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution. 

This is useful if we want to ask question about specific documents (e.g., our PDFs, a set of videos, etc). 

![](overview.jpg)

# Document Loading

![](data_loading_sources.jpg)

In [ ]:
import os
import openai

openai.api_key  = "paste-your-api-key-here"

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load all PDF
loaders = [PyPDFLoader(f"docs/{file}") for file in os.listdir('docs') if file.endswith('.pdf')]

docs = []
for loader in loaders:
    pdf_docs = loader.load()
    docs += pdf_docs

# Document Splitting

![](document_splitting.jpg)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150,
    separators=["\n\n", "(?<=\. )", "\n", " ", ""],
    is_separator_regex = True
)

splits = text_splitter.split_documents(docs)
for doc in splits:
    doc.page_content = doc.page_content.replace("\n", " ").replace('- ', "")
len(splits)

# Vectorstores and Embeddings

![](embeddings.jpg)

In [ ]:
# Use this if you have paid version of OpenAI 
# from langchain.embeddings.openai import OpenAIEmbeddings
# embedding = OpenAIEmbeddings(openai_api_key="paste-your-api-key-here")

In [ ]:
# Use this if you don't have paid version of OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
model_id = 'sentence-transformers/all-mpnet-base-v2'
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(
    model_name=model_id,
    model_kwargs=model_kwargs
)

### Vectorstores

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
persist_directory = 'chroma/'

In [ ]:
!rm -rf ./chroma  # remove old database files if any

In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

In [ ]:
vectordb.persist()

# Retrieval

![](overview.jpg)

# Question Answering

In [ ]:
llm_name = "gpt-3.5-turbo"

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key="paste-your-api-key-here")

### RetrievalQA chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5, "distance_metric": "cos"})

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

In [ ]:
result = qa_chain({"query": "Tell me something interesting about IMBH."})
result["result"]

In [ ]:
result = qa_chain({"query": "How do IMBH compare to super-massive black holes?"})
result["result"]

In [ ]:
result['source_documents']

In [ ]:
result = qa_chain({"query": "Which type of black holes are the most common?"})
result["result"]

In [ ]:
result = qa_chain({"query": "What is a typical density of IMBH?"})
result["result"]

In [ ]:
result = qa_chain({"query": "What is a typical size of IMBH?"})
result["result"]

In [ ]:
result = qa_chain({"query": "Do we have some IMBH in our galaxy?"})
result["result"]

### Prompt

Default prompts:
https://github.com/samrawal/langchain-prompts

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 8, "distance_metric": "cos"}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
result = qa_chain({"query": "How do IMBH compare to super-massive black holes?"})
result["result"]

# Chat

Recall the overall workflow for retrieval augmented generation (RAG):

![overview.jpeg](overview.jpg)

### Memory

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

### ConversationalRetrievalChain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [ ]:
question = "Tell me something interesting about IMBH."
result = qa({"question": question})
result['answer']

In [ ]:
question = "Which of these theories is the most popular among scientists?"
result = qa({"question": question})
result['answer']